In [ ]:
# Run the payment plan based off summary_transaction_2020 from step 7, after creation. Change selection to 2019 in Nayana's code.

In [ ]:
import fire
import questionary
import pandas as pd
from db_utils import get_debt_free_db_engine, get_transaction_for_user_year, get_account_for_user

In [ ]:
engine = get_debt_free_db_engine()
transaction_2019 = get_transaction_for_user_year(engine, 1, '2019')
transaction_2019['year'] = pd.DatetimeIndex(transaction_2019['transaction_date']).year
transaction_2019['month'] = pd.DatetimeIndex(transaction_2019['transaction_date']).month
summary_transaction_2019 = pd.DataFrame(round(transaction_2019.groupby(['user_id', 'card','month'], as_index=False)[('amount')].sum(),2))

In [ ]:
sumofdebt = summary_transaction_2019["amount"].sum()
user = summary_transaction_2019["user"].mean()

In [ ]:
print(f"Thank you, {user}, for choosing to consolidate your debt with our payment plan.")
print(f"Your current total debt is ${sumofdebt}.")
print("We currently have an APR on our payment plan of 5%.")
print("This will offer you easy to pay monthly payments.")
print("We have 4 options for our payment plan, please choose one of our plans listed below.")
print("Or Quit to return to our main menu.")

In [ ]:
cli_running = True

# A loop for the running of the payment plan, if they wish to view more than 1 option.
while cli_running:
    choice = questionary.select("Which option works best for you?",
        choices=["6 Months", "1 Year", "3 Year", "5 Year", "Quit"]).ask()

    if choice == "6 Months":
        payment_time = 6
        payment_amount = payment_plan_option(sumofdebt, payment_time)
            
        print(f"Your payment amount would be ${payment_amount} over {payment_time} months.")
        cli_running = payment_works()
                
    elif choice == "1 Year":
        payment_time = 12
        payment_amount = payment_plan_option(sumofdebt, payment_time)
            
        print(f"Your payment amount would be ${payment_amount} over {payment_time} months.")
        cli_running = payment_works()
            
    elif choice == "3 Year":
        payment_time = 36
        payment_amount = payment_plan_option(sumofdebt, payment_time)
            
        print(f"Your payment amount would be ${payment_amount} over {payment_time} months.")
        cli_running = payment_works()
            
    elif choice == "5 Year":
        payment_time = 60
        payment_amount = payment_plan_option(sumofdebt, payment_time)
            
        print(f"Your payment amount would be ${payment_amount} over {payment_time} months.")
        cli_running = payment_works()
        
    elif choice == "Quit":
        cli_running = False
        print("You will be returned to the main menu.")

In [ ]:
def payment_plan_option(total, time):
    # Reads in the total and the time and applies the APR to calculate the amount they will need to pay to be debt free by chosen time.
    APR = 0.05 / 12
    
    payment = total * ((APR*(pow((1+APR), time)))/(pow((1 + APR), time) - 1))

    return payment

In [ ]:
def payment_works():
    # Checks if the customer is happy with their plan.
    
    yes_no = questionary.select("Will this payment plan work for you?", choices=["Yes", "No"]).ask()
            
    if yes_no == "Yes":
        cli_running = False
        print("We thank you for choosing our consolidated payment plan.")
        print("We appreciate you as a customer.")
              
    elif yes_no == "No":
        print("We are sorry this plan doesn't work.")
        print("Please choose a different plan or choose to quit.")
        print("Or, if you'd like to set your own payment,")
        print("   please use our projection plan from our main menu.")
        cli_running = True
    
    return cli_running